In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import os
import sklearn.model_selection
import tensorboard
import pickle
%load_ext tensorboard


In [3]:
#create the dataset:
#address of the base folder:
base = "R:\\3.Masters_projects\\2021_Dominic_Rebindaine\\ROI"
#extract the name from the file name:
name_l =[]
path_l = []
for fname in os.listdir(base):
    pathto = os.path.join(base,fname)
    
    if os.path.isdir(pathto):
        path_l.append(pathto)
        split = fname.split(sep='_')
        if len(split)>2:
            name = split[1]
            name_l.append(name)
label_name, label = np.unique(name_l,return_inverse=True)

In [4]:
#define the input data:
image_size = 100 
size = image_size

In [23]:
#open the images and find their label
def data_opener(name):
    img = tf.io.decode_png(tf.io.read_file(name))
    X = tf.image.resize(img,(size,size))
    Xn = tf.image.per_image_standardization(X)
    part = tf.strings.split(name,os.sep)
    fname = part[4]
    fpart = tf.strings.split(fname,'_')
    if len(fpart)<2:
        y = -1
    else:
        el_label = fpart[1]
        y = -1
        for idx, label in enumerate(label_name):
            if label == el_label:
                y = idx
    data = (Xn,y)
    return data


        
        
def data_aug(img,label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    rnd = np.random.randint(4)
    img = tf.image.rot90(img,k=rnd)
    img = tf.image.random_hue(img,0.1)
    #stddev = tf.random.uniform(shape=[1],maxval=0.05)
    #img+= tf.random.normal((size,size,3),stddev=stddev)
    return (img,label)


def input_pipeline(ds, augmentation = False):
    ds = ds.shuffle(10000)
    ds = ds.map(data_opener)
    #filter out the unlabeled elements
    ds = ds.filter(lambda X,y: y>=0)
    if augmentation:
        ds = ds.map(data_aug)
    ds = ds.batch(32)
    ds = ds.prefetch(10)
    return ds
def destand(img_n, perc = 1,epsilon = 0.0001):
    pl,ph = np.percentile(img_n, [perc,100-perc])
    img = np.array((img_n-pl)/(epsilon+ph-pl))
    img[img>1]=1
    img[img<0]=0
    return img

In [24]:
file_names = tf.data.Dataset.list_files('R:\\3.Masters_projects\\2021_Dominic_Rebindaine\\ROI\\*\\*\\*.png')
#split the dataset now, as the loading of the image would need too long to use the list function
train, val = sklearn.model_selection.train_test_split(list(file_names),test_size= 0.2)
dt = tf.data.Dataset.from_tensor_slices(train)
dv = tf.data.Dataset.from_tensor_slices(val)

dt = input_pipeline(dt,augmentation = True)
dv = input_pipeline(dv)

In [25]:
#first test
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 50
patch_size = 10  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [512, 256]  # Size of the dense layers of the final classifier
dropout_rate = 0.5

In [ ]:
#2nd test
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 50
patch_size = 10  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 10
mlp_head_units = [720, 360]  # Size of the dense layers of the final classifier
dropout_rate = 0.7

In [12]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [13]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [14]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
data = list(dt.take(1))
image = data[0][0][0].numpy()
plt.imshow(destand(image))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([destand(image)]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy())
    plt.axis("off")


NameError: name 'dt' is not defined

<Figure size 288x288 with 0 Axes>

In [41]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
    


In [75]:
def create_vit_classifier():
    inputs = layers.Input(shape=(image_size,image_size,3))
    # Augment data.
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.8)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=dropout_rate)
    # Classify outputs.
    logits = layers.Dense(len(label_name))(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [76]:
vit_classifier = create_vit_classifier()

In [77]:
vit_classifier.summary()
vit_classifier.compile(optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
                       loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
patches_9 (Patches)             (None, None, 300)    0           input_5[0][0]                    
__________________________________________________________________________________________________
patch_encoder_3 (PatchEncoder)  (None, 100, 64)      25664       patches_9[0][0]                  
__________________________________________________________________________________________________
layer_normalization_51 (LayerNo (None, 100, 64)      128         patch_encoder_3[0][0]            
____________________________________________________________________________________________

In [30]:
savebest = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=7,
    restore_best_weights=True)

In [13]:
%tensorboard --logdir logs

In [79]:
log_dir = 'logs/test_vit_big_aug5'
history = vit_classifier.fit(dt,validation_data=dv, epochs = 50,
                    callbacks = [savebest,keras.callbacks.TensorBoard(log_dir=log_dir)],
                    use_multiprocessing=True)

Epoch 1/50
265/265 [==============================] - 255s 911ms/step - loss: 2.9487 - val_loss: 1.7796
Epoch 2/50
265/265 [==============================] - 238s 895ms/step - loss: 1.8555 - val_loss: 1.8600
Epoch 3/50
265/265 [==============================] - 233s 878ms/step - loss: 1.7222 - val_loss: 1.5105
Epoch 4/50
265/265 [==============================] - 234s 879ms/step - loss: 1.6032 - val_loss: 1.5568
Epoch 5/50
265/265 [==============================] - 237s 889ms/step - loss: 1.5317 - val_loss: 1.4142
Epoch 6/50
265/265 [==============================] - 235s 881ms/step - loss: 1.4599 - val_loss: 1.3612
Epoch 7/50
265/265 [==============================] - 226s 849ms/step - loss: 1.4006 - val_loss: 1.3475
Epoch 8/50
265/265 [==============================] - 230s 869ms/step - loss: 1.3677 - val_loss: 1.3598
Epoch 9/50
265/265 [==============================] - 239s 904ms/step - loss: 1.3205 - val_loss: 1.2530
Epoch 10/50
265/265 [==============================] - 239s 899m

## Test a pretrained model:

In [9]:
from vit_keras import vit

In [49]:
image_size = 224
model_pretrained = vit.vit_l16(
    image_size=image_size,
    activation='linear',
    pretrained=True,
    include_top=False,
    pretrained_top=False)

C:\Users\valla\Anaconda3\envs\tf-gpu\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


In [51]:
inputs = layers.Input((224,224,3))
x = model_pretrained(inputs)
out = mlp(x,[720,360],0.6)
model_full = keras.Model(inputs=inputs, outputs=out)

In [52]:
model_pretrained.trainable = False
model_full.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
vit-l16 (Functional)         (None, 1024)              303301632 
_________________________________________________________________
dense_4 (Dense)              (None, 720)               738000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 720)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 360)               259560    
_________________________________________________________________
dropout_5 (Dropout)          (None, 360)               0         
Total params: 304,299,192
Trainable params: 997,560
Non-trainable params: 303,301,632
_______________________________________

In [53]:
def data_opener_pre(name):
    img = tf.io.decode_png(tf.io.read_file(name))
    X = tf.image.resize(img,(224,224))
    Xn = tf.image.per_image_standardization(X)
    
    part = tf.strings.split(name,os.sep)
    fname = part[4]
    fpart = tf.strings.split(fname,'_')
    if len(fpart)<2:
        y = -1
    else:
        el_label = fpart[1]
        y = -1
        for idx, label in enumerate(label_name):
            if label == el_label:
                y = idx
    data = (Xn,y)
    return data
def input_pipeline_pre(ds, augmentation = False):
    ds = ds.shuffle(10000)
    ds = ds.map(data_opener_pre)
    #filter out the unlabeled elements
    ds = ds.filter(lambda X,y: y>=0)
    if augmentation:
        ds = ds.map(data_aug)
    ds = ds.batch(16)
    ds = ds.prefetch(10)
    return ds

In [54]:
dtp = tf.data.Dataset.from_tensor_slices(train)
dvp = tf.data.Dataset.from_tensor_slices(val)

dtp = input_pipeline_pre(dtp,augmentation = True)
dvp = input_pipeline_pre(dvp)

In [55]:
model_full.compile(optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
                   loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True))

In [56]:
log_dir = 'logs/test_vit_pretrained_aug5'
history = model_full.fit(dtp,validation_data=dvp, epochs = 50,
                    callbacks = [savebest,keras.callbacks.TensorBoard(log_dir=log_dir)],
                    use_multiprocessing=True)

Epoch 1/50
530/530 [==============================] - 5575s 10s/step - loss: 4.2004 - val_loss: 2.4265
Epoch 2/50
530/530 [==============================] - 1898s 4s/step - loss: 4.0179 - val_loss: 1.9667
Epoch 3/50
530/530 [==============================] - 1890s 4s/step - loss: 3.8928 - val_loss: 1.7645
Epoch 4/50
530/530 [==============================] - 1932s 4s/step - loss: 3.9110 - val_loss: 1.7525
Epoch 5/50
530/530 [==============================] - 2037s 4s/step - loss: 3.8685 - val_loss: 1.6967
Epoch 6/50
530/530 [==============================] - 1965s 4s/step - loss: 3.8645 - val_loss: 1.6894
Epoch 7/50
274/530 [==============>...............] - ETA: 5:08:53 - loss: 3.8693

UnknownError: 2 root error(s) found.
  (0) Unknown:  NewRandomAccessFile failed to Create/Open: R:\3.Masters_projects\2021_Dominic_Rebindaine\ROI\20211124_Oocystis_6mm_1\20211124_Oocystis_6mm_1_185\38_45_2.png : The network path was not found.
; Unknown error
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
  (1) Unknown:  NewRandomAccessFile failed to Create/Open: R:\3.Masters_projects\2021_Dominic_Rebindaine\ROI\20211124_Oocystis_6mm_1\20211124_Oocystis_6mm_1_185\38_45_2.png : The network path was not found.
; Unknown error
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_172313]

Function call stack:
train_function -> train_function
